# Data Aggregation

Since we have population datasets in S3, we can aggregate the data easily using AWS athena.

In this notebook, we will 

1. create a table in Athena pointing the parquets in S3
2. send a aggregation query grouping by date, hour, census_id
3. read the result csv saved in s3 bucket
4. convert it to a wide format

In [82]:
import boto3
import pandas as pd
import io

session = boto3.session.Session(profile_name='lambda')
client = session.client('athena', region_name='us-east-1')

## Create table in AWS Ahtena

In [87]:
query = """
CREATE EXTERNAL TABLE IF NOT EXISTS xpopseoul.local (
  `date` TIMESTAMP,
  `hour` int,
  `census_id` string,
  `xpop_total` int 
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
WITH SERDEPROPERTIES (
  'serialization.format' = '1'
) LOCATION 's3://xpop-seoul/monthly'
TBLPROPERTIES ('has_encrypted_data'='false');"""

In [89]:
response = client.start_query_execution(
    QueryString=query1,
    QueryExecutionContext={
        'Database': 'xpopseoul'
        },
    ResultConfiguration={
        'OutputLocation': s3_output,
        }
    )

## Aggregate data grouped by census_id, day, hour

In [64]:
query = 'select DAY_OF_WEEK(date) as day, hour, census_id, avg(xpop_total) as xpop_avg \
    from local \
    group by DAY_OF_WEEK(date), hour, census_id'
s3_bucket = "xpop-seoul"
folder = 'aggregated'
s3_output = "s3://{}/{}".format(s3_bucket, folder)

In [28]:
response = client.start_query_execution(
    QueryString=query,
    QueryExecutionContext={
        'Database': 'xpopseoul'
        },
    ResultConfiguration={
        'OutputLocation': s3_output,
        }
    )

In [63]:
result_csv = "{}/{}.csv".format(s3_output, response['QueryExecutionId'])

In [67]:
s3 = session.resource('s3')
csv = '../data/interim/aggregated.csv'
s3.Bucket(s3_bucket).download_file(
    '{}/{}.csv'.format(folder, response['QueryExecutionId']), 
    csv)

In [68]:
df = pd.read_csv(csv)
df.head()

,day,hour,census_id,xpop_avg
0,1,15,1101056030701,46.802198
1,1,13,1105056020018,107.120879
2,1,15,1105061010024,162.560440
3,1,5,1105061010027,650.263736
4,1,12,1107072030101,162.692308


In [69]:
df.dtypes

day            int64
hour           int64
census_id      int64
xpop_avg     float64
dtype: object

##  make df wide

In [70]:
df['timeslot'] = df['day'].map('D{}'.format).str.cat(df['hour'].map('T{:02d}'.format))

In [75]:
df_wide = df.pivot(index='census_id', columns='timeslot', values='xpop_avg')
df_wide.head()

timeslot,D1T00,D1T01,D1T02,D1T03,D1T04,D1T05,D1T06,D1T07,D1T08,D1T09,...,D7T14,D7T15,D7T16,D7T17,D7T18,D7T19,D7T20,D7T21,D7T22,D7T23
census_id,,,,,,,,,,,,,,,,,,,,,
1101053010001,397.934066,391.835165,390.043956,387.483516,360.747253,354.384615,358.450549,341.043956,315.505495,302.780220,...,364.304348,383.369565,364.467391,371.532609,376.163043,369.369565,364.456522,339.130435,331.652174,388.826087
1101053010002,1180.208791,1179.494505,1164.747253,1154.934066,1166.978022,1213.472527,1314.571429,1508.901099,1650.439560,1723.912088,...,1751.717391,1737.141304,1707.728261,1745.717391,1815.956522,1799.532609,1780.184783,1655.880435,1502.130435,1200.336957
1101053010003,1258.043956,1261.208791,1249.626374,1249.593407,1271.824176,1288.703297,1302.637363,1535.791209,1573.351648,1590.483516,...,1979.434783,1982.836957,2021.956522,1999.293478,1894.586957,1690.717391,1452.326087,1377.336957,1331.173913,1281.760870
1101053010004,1222.637363,1139.109890,1152.857143,1135.670330,1161.549451,1248.065934,1479.461538,2218.461538,3026.835165,3513.923077,...,3326.510870,3395.684783,3420.282609,3273.489130,2984.793478,2663.326087,2379.967391,2168.771739,1945.706522,1325.043478
1101053010005,833.373626,827.703297,793.373626,794.714286,791.681319,865.780220,1072.527473,1527.197802,2511.065934,2848.054945,...,2487.119565,2474.152174,2333.043478,2088.760870,1890.358696,1847.782609,1729.750000,1573.869565,1433.891304,873.423913


In [81]:
df_wide.to_csv("../data/interim/aggregate_wide.csv")